# CARGA DE LIBRERIAS

In [ ]:
import database
import eda


# DESCARGA DE LOS DATOS

In [ ]:
database.initialize_database()
cats = database.download_data()

# CARGA DATOS

In [ ]:
database.upload_to_db(cats)

# DESCARGA DE DATOS A DATAFRAME

In [ ]:
df = database.get_dataframe()

# EDA

In [ ]:
eda.data_resume_info(df=df)